In [2]:
import pandas as pd

In [4]:
pd.options.display.max_colwidth = None

In [9]:
df =pd.read_csv('./data/eval_results.csv')
df.head()

,question,answer,custom_gpt2_answer,oob_gpt2_answer,bert_score_custom_gpt2,bert_score_oob_gpt2,reward
0,"i have a few symptoms like the stomachache, congestion, and diarrhea but no fever. is it possible i have the virus?","stomach troubles aren't a common symptom of the coronavirus, but a fever is a key symptom, so it's unlikely that you have it. however, if you have any questions, call ahead to your doctor and make an appointment.",yes! while we are still learning about how covid-2019 affects people’s health more information can be found on cdc website (pdf),yes! you may be able to get covid-19 from eating raw meat or poultry that has been contaminated with sarsaparic acid (sARS) in your food source for at least 20 days after exposure so there are some signs of illness associated if this happens during those 30 day period when people eat foods high on antibiotics such as chicken breast soup instead,0.830807,0.819558,0.011249
1,what if my time off is not approved and i don’t come to work?,you will be treated just as you would if you did not show up for work any other time. employees failing to come to work without approved leave time may be subject to discipline per the applicable collective bargaining agreement or compensation plan.,"you can volunteer at a food bank or other community organization. make sure they have the supplies where their workers are, as well your insurance information so that when someone in an essential business like grocery stores becomes ill it doesn't affect them financially but cannot donate money directly toward those who need help with groceries during this pandemic (or else)","you can contact your employer or the local fire department for assistance with contacting their office regarding a potential employee who may be unable due leave. they will provide information about how long it takes them until an individual has been in place, such as when hours are reduced (e-mailing) onsite/out of state by telephone call; where employees should go out during this period without interruption while working from home but do so at least",0.817220,0.832336,-0.015117
2,where can i find more information about animals and covid-19?,"for more information, check out the following websites: cdc covid-19: pets and other animals cdc: frequently asked questions, covid-19 and animals cdc: interim guidance for public health professionals managing people with covid-19 in home care and isolation who have pets or other animals cdc: healthy pets, healthy people cdc: covid-19 daily life and coping, if you have animals avma: interim recommendations for intake of companion animals from households where humans with covid-19 are present","the centers for disease control (cdc) is constantly updating its website with resources to help people understand this new virus. if you or someone in your family has questions, reach out via email at afnrsa@vdh3sorg/coronavirus—or call 804‑234–7364 –and ask them on facebook messenger! they’ll answer any queries that come their way through these chats; otherwise we encourage anyone who's interested by reading what cdc guidance applies here.","the cdc has a list of animal health organizations that are working on prevention, treatment strategies for people with or without symptoms associatedcov infection. see https://www/cdphs’vegetarianscoronavirus/.",0.798086,0.806660,-0.008574
3,what precautions should i take during travel?,"during travel, everyone should clean hands frequently, cough or sneeze into a bent elbow or tissue, and try to maintain a physical distance of at least one metre from others. travelers should follow the recommendations of the travel authorities regarding policies in the airport and of the airline for the flight.","if you have been in close contact with someone suspected of having covid-19, or are feeling unwell (like coughing), avoid touching your face. this includes phones and other electronic devices that might be used by people who were not well before the outbreak began